In [1]:
import sys
print(sys.version)
print(sys.executable)
import pkgutil
print(dir(pkgutil))

3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
D:\anaconda\python.exe
['ImpImporter', 'ImpLoader', 'ModuleInfo', 'ModuleType', '_NAME_PATTERN', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_get_spec', '_import_imp', '_iter_file_finder_modules', 'extend_path', 'find_loader', 'get_data', 'get_importer', 'get_loader', 'importlib', 'iter_importer_modules', 'iter_importers', 'iter_modules', 'iter_zipimport_modules', 'namedtuple', 'os', 're', 'read_code', 'resolve_name', 'simplegeneric', 'sys', 'walk_packages', 'warnings', 'zipimport', 'zipimporter']


In [2]:
import collections
import random
import math
import os
import shutil
import pandas as pd
import torch 
import torchvision
from torch import nn
from d2l import torch as d2l

In [3]:
# 示例路径
dest_dir = r'C:\Users\29180\Desktop\超声乳腺影像的BIRADS分类及特征识别\train_valid_test'
train_dir = r'C:\Users\29180\Desktop\超声乳腺影像的BIRADS分类及特征识别\超声乳腺训练数据+测试集A+说明文档\乳腺分类训练数据集\train'
test_dir = r'C:\Users\29180\Desktop\超声乳腺影像的BIRADS分类及特征识别\超声乳腺训练数据+测试集A+说明文档\test_A\乳腺分类测试集A\A'

In [4]:
def copy_file(src_file, dest_dir):
    """将文件复制到目标目录"""
    os.makedirs(dest_dir, exist_ok=True)
    shutil.copy(src_file, dest_dir)

In [5]:
def organize_train_valid(train_dir, dest_dir, valid_ratio):
    """组织训练集和验证集"""
    # 创建目标目录结构
    train_dest_dir = os.path.join(dest_dir, 'train')
    train_valid_dest_dir = os.path.join(dest_dir, 'train_valid')
    valid_dest_dir = os.path.join(dest_dir, 'valid')
    
    os.makedirs(train_dest_dir, exist_ok=True)
    os.makedirs(train_valid_dest_dir, exist_ok=True)
    os.makedirs(valid_dest_dir, exist_ok=True)
    
    for class_name in os.listdir(train_dir):
        class_images_dir = os.path.join(train_dir, class_name, 'images')
        if not os.path.exists(class_images_dir):
            continue
        
        images = os.listdir(class_images_dir)
        num_valid = max(1, math.floor(len(images) * valid_ratio))
        
        # 随机选择部分图像作为验证集
        valid_images = random.sample(images, num_valid)
        
        for image_name in images:
            src_image_path = os.path.join(class_images_dir, image_name)
            # 复制到 train_valid 文件夹
            copy_file(src_image_path, os.path.join(train_valid_dest_dir, class_name))
            # 复制到 valid 或 train 文件夹
            if image_name in valid_images:
                copy_file(src_image_path, os.path.join(valid_dest_dir, class_name))
            else:
                copy_file(src_image_path, os.path.join(train_dest_dir, class_name))

In [6]:
def organize_test(test_dir, dest_dir):
    """组织测试集"""
    test_dest_dir = os.path.join(dest_dir, 'test', 'unknown')
    os.makedirs(test_dest_dir, exist_ok=True)
    
    for class_name in os.listdir(test_dir):
        class_images_dir = os.path.join(test_dir, class_name, 'images')
        if not os.path.exists(class_images_dir):
            continue
        
        for image_name in os.listdir(class_images_dir):
            src_image_path = os.path.join(class_images_dir, image_name)
            copy_file(src_image_path, test_dest_dir)

In [7]:
def reorg_data(train_dir, test_dir, dest_dir, valid_ratio=0.1):
    """调用上面的函数，组织训练集、验证集和测试集"""
    organize_train_valid(train_dir, dest_dir, valid_ratio)
    organize_test(test_dir, dest_dir)
    
reorg_data(train_dir, test_dir, dest_dir, valid_ratio=0.1)

In [8]:
# 图像增广
transform_train = torchvision.transforms.Compose([
    torchvision.transforms.Resize(40),  # 缩放到 40x40 像素
    torchvision.transforms.RandomResizedCrop(32, scale=(0.64, 1.0), ratio=(1.0, 1.0)),  # 随机裁剪并缩放到 32x32
    torchvision.transforms.RandomHorizontalFlip(),  # 随机水平翻转
    torchvision.transforms.RandomVerticalFlip(),  # 随机垂直翻转
    torchvision.transforms.RandomRotation(degrees=30),  # 随机旋转 -30 到 30 度
    torchvision.transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 随机平移
    torchvision.transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # 随机网格畸变
    torchvision.transforms.ToTensor(),  # 转换为张量
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])  # 归一化
])

transform_test = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914,0.4822,0.4465],
                                    [0.2023,0.1994,0.2010]) ])

In [9]:
# 使用 ImageFolder 加载数据集
train_ds, train_valid_ds = [
    torchvision.datasets.ImageFolder(
        os.path.join(dest_dir, folder),
        transform=transform_train) for folder in ['train', 'train_valid']
]

valid_ds, test_ds = [
    torchvision.datasets.ImageFolder(
        os.path.join(dest_dir, folder),
        transform=transform_test) for folder in ['valid', 'test']
]
